# <ins>**PreCog recruitment programming task-2: Phrase and Sentence similarity**</ins>
## Phrase Similarity:

In [ ]:
!pip install gensim datasets scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00


##We will be using the glove-twitter-25 pre-trained embedding for this task

In [ ]:
import gensim
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
glove_vectors['hello']

array([-0.77069  ,  0.12827  ,  0.33137  ,  0.0050893, -0.47605  ,
       -0.50116  ,  1.858    ,  1.0624   , -0.56511  ,  0.13328  ,
       -0.41918  , -0.14195  , -2.8555   , -0.57131  , -0.13418  ,
       -0.44922  ,  0.48591  , -0.6479   , -0.84238  ,  0.61669  ,
       -0.19824  , -0.57967  , -0.65885  ,  0.43928  , -0.50473  ],
      dtype=float32)

##Lets download the dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("PiC/phrase_similarity")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 7004
    })
    validation: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2000
    })
})


In [ ]:
import pandas as pd
train_df = dataset['train'].to_pandas()
val_df = dataset['validation'].to_pandas()
test_df = dataset['test'].to_pandas()
train_df.head()

,phrase1,phrase2,sentence1,sentence2,label,idx
0,newly formed camp,recently made encampment,newly formed camp is released from the membran...,recently made encampment is released from the ...,0,0
1,one data,a particular statistic,"According to one data, in 1910, on others – in...","According to a particular statistic, in 1910, ...",1,1
2,particular structure,specific edifice,Note that Fact 1 does not assume any particula...,Note that Fact 1 does not assume any specific ...,0,2
3,involved people,participating individuals,Assessment-Center are usually group-processes ...,Assessment-Center are usually group-processes ...,1,3
4,different cross,opposing inquiries,"At the end of the 1980s, a different cross had...","At the end of the 1980s, a opposing inquiries ...",0,4


In [ ]:
train_df = train_df.dropna()
test_df = test_df.dropna()
val_df = val_df.dropna()

## Extracting the phrase columns...

In [ ]:
train_df = train_df[['phrase1','phrase2','label']]
test_df = test_df[['phrase1','phrase2','label']]
val_df = val_df[['phrase1','phrase2','label']]

In [ ]:
train_df.head()

,phrase1,phrase2,label
0,newly formed camp,recently made encampment,0
1,one data,a particular statistic,1
2,particular structure,specific edifice,0
3,involved people,participating individuals,1
4,different cross,opposing inquiries,0


## Tokenizing each phrase-pair...

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
train_phrase1 = train_df['phrase1'].apply(word_tokenize)
train_phrase2 = train_df['phrase2'].apply(word_tokenize)
val_phrase1 = val_df['phrase1'].apply(word_tokenize)
val_phrase2 = val_df['phrase2'].apply(word_tokenize)
test_phrase1 = test_df['phrase1'].apply(word_tokenize)
test_phrase2 = test_df['phrase2'].apply(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
train_phrase1, test_phrase2

(0         [newly, formed, camp]
 1                   [one, data]
 2       [particular, structure]
 3            [involved, people]
 4            [different, cross]
                  ...           
 6999          [similar, notice]
 7000               [color, map]
 7001              [dutch, tool]
 7002       [secondary, concern]
 7003              [full, order]
 Name: phrase1, Length: 7004, dtype: object,
 0           [posture, while, jumping]
 1        [correlating, music, single]
 2                [extended, sections]
 3              [spacious, collection]
 4                 [complete, defense]
                     ...              
 1995    [what, is, really, going, on]
 1996              [a, singular, tier]
 1997                   [Full, sample]
 1998     [at, play, ,, human, beings]
 1999               [General, Service]
 Name: phrase2, Length: 2000, dtype: object)

In [ ]:
train_phrase1 = train_phrase1.tolist()
train_phrase2 = train_phrase2.tolist()
val_phrase1 = val_phrase1.tolist()
val_phrase2 = val_phrase2.tolist()
test_phrase1 = test_phrase1.tolist()
test_phrase2 = test_phrase2.tolist()

In [ ]:
train_phrase1, test_phrase2

([['newly', 'formed', 'camp'],
  ['one', 'data'],
  ['particular', 'structure'],
  ['involved', 'people'],
  ['different', 'cross'],
  ['wide', 'majority'],
  ['major', 'criticism'],
  ['robust', 'selection'],
  ['huge', 'round'],
  ['general', 'point'],
  ['power', 'imbalance'],
  ['unusual', 'period'],
  ['individual', 'body'],
  ['good', 'prospects'],
  ['one', 'argument'],
  ['abrupt', 'release'],
  ['single', 'end'],
  ['former', 'charge'],
  ['smooth', 'one'],
  ['fellow', 'division'],
  ['common', 'source'],
  ['common', 'focus'],
  ['performance', 'fee'],
  ['additional', 'use'],
  ['new', 'practice'],
  ['one', 'notch'],
  ['active', 'counterpart'],
  ['material', 'obstacles'],
  ['administrative', 'setting'],
  ['much', 'tension'],
  ['potential', 'interest'],
  ['main', 'drive'],
  ['state', 'sponsor'],
  ['representative', 'element'],
  ['growing', 'line'],
  ['original', 'chain'],
  ['similar', 'record'],
  ['trail', 'name'],
  ['program', "'s", 'design'],
  ['recent', 'ch

## Converting tokens to vector representations using GloVe embeddings

In [ ]:
import numpy as np
def get_glove_vectors(words):
    vectors = []
    for word in words:
        try:
            vector = glove_vectors[word]
            vectors.append(vector)
        except KeyError:
            vector = np.zeros_like(glove_vectors['sample'])  # You can replace 'sample' with any existing word in your model
        vectors.append(vector)
    return vectors

In [ ]:
train_phrase1_vecs = [get_glove_vectors(phrase) for phrase in train_phrase1]
train_phrase2_vecs = [get_glove_vectors(phrase) for phrase in train_phrase2]
val_phrase1_vecs = [get_glove_vectors(phrase) for phrase in val_phrase1]
val_phrase2_vecs = [get_glove_vectors(phrase) for phrase in val_phrase2]
test_phrase1_vecs = [get_glove_vectors(phrase) for phrase in test_phrase1]
test_phrase2_vecs = [get_glove_vectors(phrase) for phrase in test_phrase2]

In [ ]:
def avg_phrase_vecs(phrase1_vecs, phrase2_vecs):
  avg_phrase1_vec = []
  avg_phrase2_vec = []
  for phrase1_vec, phrase2_vec in zip(phrase1_vecs, phrase2_vecs):
      avg_phrase1_vec.append(np.mean(phrase1_vec, axis=0))
      avg_phrase2_vec.append(np.mean(phrase2_vec, axis=0))
  return avg_phrase1_vec, avg_phrase2_vec
train_avg_phrase1_vec,train_avg_phrase2_vec = avg_phrase_vecs(train_phrase1_vecs, train_phrase2_vecs)
val_avg_phrase1_vec,val_avg_phrase2_vec = avg_phrase_vecs(val_phrase1_vecs, val_phrase2_vecs)
test_avg_phrase1_vec,test_avg_phrase2_vec = avg_phrase_vecs(test_phrase1_vecs, test_phrase2_vecs)

In [ ]:
train_avg_phrase1_vec

[array([-0.4404734 ,  0.21458156,  0.04497066, -0.9977067 ,  0.17158665,
        -0.65262336,  0.307865  , -0.7031333 , -0.12009334, -0.43618998,
         0.47066268,  0.3792    , -2.5997336 ,  0.7824967 , -0.18108337,
         0.12047333,  0.43944   ,  0.8021066 ,  0.5654734 , -0.270756  ,
        -0.6019    , -0.22029431, -0.30226734, -0.564116  , -0.03524667],
       dtype=float32),
 array([ 7.81584978e-01,  2.55919993e-01,  1.06570005e-01, -2.81027496e-01,
         6.52350008e-01, -1.56392515e-01,  9.06746030e-01,  1.53167501e-01,
         5.15859962e-01,  1.02999806e-03,  2.96677530e-01, -4.32720006e-01,
        -4.14444971e+00,  2.32964009e-01, -1.49585009e-01,  6.79149926e-02,
        -5.22449985e-02,  2.95445025e-01, -1.05370499e-01, -6.55325055e-01,
        -5.62962472e-01, -6.56795025e-01, -2.37989992e-01, -1.92800015e-02,
        -8.44094992e-01], dtype=float32),
 array([ 0.16137   , -0.34990498, -0.550915  , -0.02652   ,  0.9100599 ,
         0.06372205,  0.1453345 , -0.858

## Averaging the word-vectors in each phrase...

In [ ]:
train_avged = [np.mean([train_avg_phrase1_vec[i],train_avg_phrase2_vec[i]],axis=0) for i in range(len(train_avg_phrase1_vec))]
val_avged = [np.mean([val_avg_phrase1_vec[i],val_avg_phrase2_vec[i]],axis=0) for i in range(len(val_avg_phrase1_vec))]
test_avged = [np.mean([test_avg_phrase1_vec[i],test_avg_phrase2_vec[i]],axis=0) for i in range(len(test_avg_phrase1_vec))]
print(len(train_avged),len(val_avged))

7004 1000


In [ ]:
X_train,y_train = train_avged,train_df['label']
X_val,y_val = val_avged,val_df['label']
X_test,y_test = test_avged,test_df['label']

## Testing out different models with hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

models = {
    'Logistic Regression': (LogisticRegression(), {'C': [0.0001,0.001, 0.01, 0.1, 1, 10, 100, 100000]}),
    'SGD Classifier': (SGDClassifier(), {'alpha': [0.0000001, 0.001, 0.01, 0.1, 1, 10]}),
    # 'Random Forest': (RandomForestClassifier(), {'n_estimators': [50, 100, 200]}),
    'MLP Classifier': (MLPClassifier(max_iter=3000), {'hidden_layer_sizes': [(50,), (100,), (50, 50)]}),
    # 'SVM': (SVC(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
    # 'KNN': (KNeighborsClassifier(), {'n_neighbors': [3, 5, 7]}),
    # 'KMeans': (KMeans(), {'n_clusters': [3, 5, 7]})
}


for name, (model, param_grid) in models.items():
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    print(f"Best hyperparameters for {name}: {grid_search.best_params_}")

    best_model = grid_search.best_estimator_
    y_val_pred = best_model.predict(X_val)

    accuracy = accuracy_score(y_val, y_val_pred)
    report = classification_report(y_val, y_val_pred)

    print(f"Validation Accuracy for {name}: {accuracy}")
    print("Validation Classification Report:\n", report)
    print("\n" + "="*50 + "\n")


Best hyperparameters for Logistic Regression: {'C': 0.001}
Validation Accuracy for Logistic Regression: 0.484
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.49      0.49       500
           1       0.48      0.48      0.48       500

    accuracy                           0.48      1000
   macro avg       0.48      0.48      0.48      1000
weighted avg       0.48      0.48      0.48      1000



Best hyperparameters for SGD Classifier: {'alpha': 1}
Validation Accuracy for SGD Classifier: 0.5
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       500
           1       0.50      1.00      0.67       500

    accuracy                           0.50      1000
   macro avg       0.25      0.50      0.33      1000
weighted avg       0.25      0.50      0.33      1000





/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Choosing MLP classifier and training the model

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_classifier = MLPClassifier(random_state=1, max_iter=1000, learning_rate_init=1e-7)

mlp_classifier.fit(X_train, y_train)

y_val_pred = mlp_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_val_pred)
report = classification_report(y_test, y_val_pred)
print("MLP Classifier Accuracy:", accuracy)
print("MLP Classifier Classification Report:\n", report)

MLP Classifier Accuracy: 0.4915
MLP Classifier Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.06      0.11      1000
           1       0.50      0.92      0.64      1000

    accuracy                           0.49      2000
   macro avg       0.47      0.49      0.38      2000
weighted avg       0.47      0.49      0.38      2000



#Feature Engineering

## The above method just averaged both the phrase-pair's embeddings. In this method we will extract various features from individual phrase embeddings to train the model



In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Extracting cosine-similarity, euclidean-similarity and POS-tag overlaps for all the phrase-pair embeddings

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def calculate_cosine_similarity(vec1, vec2):
    return cosine_similarity(vec1, vec2)[0][0]

def get_pos_tags(phrase):
    tokens = word_tokenize(phrase)
    return [tag for _, tag in pos_tag(tokens)]

def pad_or_truncate(vector, target_length):
    current_length = len(vector)
    if current_length < target_length:
        return np.pad(vector, (0, target_length - current_length), 'constant')
    elif current_length > target_length:
        return vector[:target_length]
    else:
        return vector

features = []

for phrase1, phrase2, vec1, vec2 in zip(train_phrase1, train_phrase2, train_avg_phrase1_vec,train_avg_phrase2_vec):
    # vec1 = np.array(vec1)
    # vec2 = np.array(vec2)

    cosine_sim = calculate_cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))
    target_length = max(len(vec1), len(vec2))
    vec1_padded = pad_or_truncate(vec1, target_length)
    vec2_padded = pad_or_truncate(vec2, target_length)

    pos_tags1 = get_pos_tags(" ".join(phrase1))
    pos_tags2 = get_pos_tags(" ".join(phrase2))

    pos_overlap_count = len(set(pos_tags1) & set(pos_tags2))
    euclidean_dist = np.linalg.norm(vec1 - vec2)
    features.append([cosine_sim, pos_overlap_count, euclidean_dist])


columns = ['Cosine_Similarity', 'POS_Overlap_Count', 'Euclidean_dist']
df_features = pd.DataFrame(features, columns=columns)

df_features['Label'] = train_df['label']

print(df_features.head())


   Cosine_Similarity  POS_Overlap_Count  Euclidean_dist  Label
0           0.885689                  2        1.688101      0
1           0.826150                  1        2.637402      1
2           0.738795                  2        2.600387      0
3           0.798528                  1        3.179536      1
4           0.502303                  0        4.270213      0


## Training the MLP classifier that performs slightly better

In [ ]:
X = df_features.drop('Label', axis=1)
y = df_features['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42, learning_rate_init=1e-7)
mlp_classifier.fit(X_train, y_train)

y_pred = mlp_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred))

Accuracy: 0.5010706638115632
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       699
           1       0.50      1.00      0.67       702

    accuracy                           0.50      1401
   macro avg       0.25      0.50      0.33      1401
weighted avg       0.25      0.50      0.33      1401



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Sentence Similarity

Note: for sentence similarity, I used the exact same method as above, i.e, same features extraction and training on MLP classifier

In [ ]:
from datasets import load_dataset
dataset = load_dataset("paws", "labeled_final")

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [ ]:
import pandas as pd
train_df = dataset['train'].to_pandas()
val_df = dataset['validation'].to_pandas()
test_df = dataset['test'].to_pandas()
train_df.head()

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1


In [ ]:
train_df = train_df[['sentence1', 'sentence2', 'label']]
val_df = val_df[['sentence1', 'sentence2', 'label']]
test_df = test_df[['sentence1', 'sentence2', 'label']]
train_df.head()

,sentence1,sentence2,label
0,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1


In [ ]:
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
train_sentence_1 = train_df['sentence1'].apply(word_tokenize)
train_sentence_2 = train_df['sentence2'].apply(word_tokenize)
train_labels = train_df['label']

val_sentence_1 = val_df['sentence1'].apply(word_tokenize)
val_sentence_2 = val_df['sentence2'].apply(word_tokenize)
val_labels = val_df['label']

test_sentence_1 = test_df['sentence1'].apply(word_tokenize)
test_sentence_2 = test_df['sentence2'].apply(word_tokenize)
test_labels = test_df['label']

In [ ]:
train_sentence_1 = train_sentence_1.tolist()
train_sentence_2 = train_sentence_2.tolist()
train_labels = train_labels.tolist()

val_sentence_1 = val_sentence_1.tolist()
val_sentence_2 = val_sentence_2.tolist()
val_labels = val_labels.tolist()

test_sentence_1 = test_sentence_1.tolist()
test_sentence_2 = test_sentence_2.tolist()
test_labels = test_labels.tolist()

print(type(train_sentence_1))

<class 'list'>


In [ ]:
train_sentence1_vecs = [get_glove_vectors(sentence) for sentence in train_sentence_1]
train_sentence2_vecs = [get_glove_vectors(sentence) for sentence in train_sentence_2]

val_sentence1_vecs = [get_glove_vectors(sentence) for sentence in val_sentence_1]
val_sentence2_vecs = [get_glove_vectors(sentence) for sentence in val_sentence_2]

test_sentence1_vecs = [get_glove_vectors(sentence) for sentence in test_sentence_1]
test_sentence2_vecs = [get_glove_vectors(sentence) for sentence in test_sentence_2]
train_phrase1_vecs[:10]

[[array([-0.53881 ,  0.065865,  0.60327 , -1.3862  ,  0.95157 , -0.87058 ,
          0.57306 , -0.83524 , -0.19705 , -0.59224 ,  1.0369  ,  0.56159 ,
         -2.3993  , -0.15901 , -0.88085 , -0.58489 ,  1.4003  ,  1.5198  ,
          0.6508  , -0.3346  , -0.80028 , -1.1062  , -0.90632 , -0.56776 ,
         -0.49658 ], dtype=float32),
  array([-0.53881 ,  0.065865,  0.60327 , -1.3862  ,  0.95157 , -0.87058 ,
          0.57306 , -0.83524 , -0.19705 , -0.59224 ,  1.0369  ,  0.56159 ,
         -2.3993  , -0.15901 , -0.88085 , -0.58489 ,  1.4003  ,  1.5198  ,
          0.6508  , -0.3346  , -0.80028 , -1.1062  , -0.90632 , -0.56776 ,
         -0.49658 ], dtype=float32),
  array([-0.5766   , -0.0071803, -0.54377  , -0.79286  , -0.25932  ,
         -0.82164  , -0.021945 , -1.5159   , -0.68476  , -0.18548  ,
          0.33921  ,  0.89639  , -2.2411   ,  1.4979   , -0.6924   ,
          0.75014  ,  0.29715  ,  1.0643   ,  1.5124   , -0.40051  ,
         -0.13662  ,  0.45798  ,  0.081322 , -0.03

In [ ]:
train_avg_sentence1_vec, train_avg_sentence2_vec = avg_phrase_vecs(train_sentence1_vecs, train_sentence2_vecs)
val_avg_sentence1_vec, val_avg_sentence2_vec = avg_phrase_vecs(val_sentence1_vecs, val_sentence2_vecs)
test_avg_sentence1_vec, test_avg_sentence2_vec = avg_phrase_vecs(test_sentence1_vecs, test_sentence2_vecs)

train_avg_sentence1_vec[:10]

[array([ 0.05934074,  0.04721346,  0.05227403,  0.03171845, -0.31027105,
        -0.04501289,  0.48406413,  0.2303925 , -0.09710726,  0.09401384,
         0.14727446, -0.18113974, -3.6653843 , -0.01970245,  0.05662441,
        -0.11316857,  0.00572596, -0.05628933, -0.37963668, -0.5178145 ,
        -0.08260925,  0.05720049, -0.23603867, -0.05943429, -0.15756379],
       dtype=float32),
 array([ 0.20812045, -0.05882547,  0.05773924, -0.15805553, -0.12952822,
         0.01012516,  1.0078869 ,  0.13132384, -0.16675001, -0.06793809,
         0.1476246 ,  0.4235316 , -3.4041922 ,  0.38453928,  0.25817508,
        -0.23773001,  0.05208153,  0.04572605, -0.34352234, -0.01161616,
        -0.47141606,  0.16837767, -0.06329185, -0.13103308,  0.049348  ],
       dtype=float32),
 array([ 0.376217  , -0.04576193, -0.325261  , -0.31453124,  0.25267673,
        -0.01701137,  0.7092152 , -0.3610432 , -0.00889839, -0.10560131,
         0.04951607,  0.04798057, -3.389512  ,  0.26788962,  0.36777723,
   

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def calculate_cosine_similarity(vec1, vec2):
    return cosine_similarity(vec1, vec2)[0][0]

def get_pos_tags(phrase):
    tokens = word_tokenize(phrase)
    return [tag for _, tag in pos_tag(tokens)]

def pad_or_truncate(vector, target_length):
    current_length = len(vector)
    if current_length < target_length:
        return np.pad(vector, (0, target_length - current_length), 'constant')
    elif current_length > target_length:
        return vector[:target_length]
    else:
        return vector

features = []

for phrase1, phrase2, vec1, vec2 in zip(train_sentence_1, train_sentence_2, train_avg_sentence1_vec, train_avg_sentence2_vec):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    cosine_sim = calculate_cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))
    target_length = max(len(vec1), len(vec2))
    vec1_padded = pad_or_truncate(vec1, target_length)
    vec2_padded = pad_or_truncate(vec2, target_length)

    pos_tags1 = get_pos_tags(" ".join(phrase1))
    pos_tags2 = get_pos_tags(" ".join(phrase2))

    pos_overlap_count = len(set(pos_tags1) & set(pos_tags2))

    features.append([cosine_sim, pos_overlap_count])


columns = ['Cosine_Similarity', 'POS_Overlap_Count']
df_features = pd.DataFrame(features, columns=columns)

df_features['Label'] = train_df['label']

print(df_features.head())

   Cosine_Similarity  POS_Overlap_Count  Label
0           0.998575                 12      0
1           0.999626                  9      1
2           0.996356                  9      0
3           0.990939                 10      1
4           0.989759                  7      1


In [ ]:
X = df_features.drop('Label', axis=1)
y = df_features['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42, learning_rate_init=1e-2)
mlp_classifier.fit(X_train, y_train)

y_pred = mlp_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred))

Accuracy: 0.5615828357453699
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      5549
           1       0.00      0.00      0.00      4332

    accuracy                           0.56      9881
   macro avg       0.28      0.50      0.36      9881
weighted avg       0.32      0.56      0.40      9881



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# <ins>**Bonus Task**</ins>

##SIMBArt-Phrase: Phrase similarity detection BERT model trained on PiC(phrase-in-context) dataset

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

model = TFAutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(['hello world','How are you'], padding=True, truncation=True, return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0],
       [ 101, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
output = model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 5, 768), dtype=float32, numpy=
array([[[-0.1688832 ,  0.13606352, -0.13940024, ..., -0.6251128 ,
          0.05217276,  0.36714536],
        [-0.3632746 ,  0.14121912,  0.87998706, ...,  0.10432965,
          0.28875774,  0.37267962],
        [-0.6985944 , -0.698797  ,  0.06450319, ..., -0.22103733,
          0.00986863, -0.5939796 ],
        [ 0.8309826 ,  0.12366714, -0.15119047, ...,  0.1030959 ,
         -0.6779267 , -0.26285207],
        [-0.40266573, -0.01928236,  0.57325023, ..., -0.20656858,
          0.02338598,  0.20126319]],

       [[ 0.0774727 ,  0.04501351, -0.16765113, ..., -0.33463413,
          0.16396382,  0.12080589],
        [-0.02718062, -0.7117558 , -0.29173565, ..., -0.12723008,
          0.6787842 , -0.53435344],
        [ 0.64420474, -0.98045236, -0.03809451, ..., -0.6010789 ,
          0.16867961, -0.49630678],
        [ 0.01914815, -0.7620821 ,  0.588272  , ..., -0.32224914

In [ ]:
from datasets import load_dataset
dataset = load_dataset("PiC/phrase_similarity")
dataset

DatasetDict({
    train: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 7004
    })
    validation: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2000
    })
})

## Combining the phrase and sentence columns from the dataset

In [ ]:
def combine_columns(example):
    return {'text': f"[{example['phrase1']}] [{example['phrase2']}] [{example['sentence1']}] [{example['sentence2']}]"}

dataset['train'] = dataset['train'].map(combine_columns)
dataset['validation'] = dataset['validation'].map(combine_columns)
dataset['test'] = dataset['test'].map(combine_columns)

dataset['train'] = dataset['train'].remove_columns(['phrase1', 'phrase2', 'sentence1', 'sentence2','idx'])
dataset['validation'] = dataset['validation'].remove_columns(['phrase1', 'phrase2', 'sentence1', 'sentence2','idx'])
dataset['test'] = dataset['test'].remove_columns(['phrase1', 'phrase2', 'sentence1', 'sentence2','idx'])


Map:   0%|          | 0/7004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][7]

{'label': 1,
 'text': '[robust selection] [powerful choice] [Both tests have reliable power to find soft sweeps for robust selection and a high starting frequency (5–20%) of the selected allele.] [Both tests have reliable power to find soft sweeps for powerful choice and a high starting frequency (5–20%) of the selected allele.]'}

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

## Encoding the dataset with the tokenizer

In [ ]:
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)
dataset_encoded

Map:   0%|          | 0/7004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7004
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

## Converting the dataset to tensorflow format

In [ ]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

dataset_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for val set ...
val_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['validation'][:])
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 138), dtype=int64, numpy=
array([[ 101, 1031, 2028, ...,    0,    0,    0],
       [ 101, 1031, 2701, ...,    0,    0,    0],
       [ 101, 1031, 2178, ...,    0,    0,    0],
       ...,
       [ 101, 1031, 6204, ...,    0,    0,    0],
       [ 101, 1031, 4319, ...,    0,    0,    0],
       [ 101, 1031, 2434, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(64, 138), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 138), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[1 1 1 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1

## Subclassing the keras API to train the model with activation as softmax

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

## Initialising the classifier with AdamW as optimizer and sparse categorical cross entropy as the loss function

In [ ]:
classifier = BERTForClassification(model, num_classes=2)

classifier.compile(
    optimizer=tf.keras.optimizers.AdamW(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

## Finally training it

In [ ]:
history1 = classifier.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
110/110 [==============================] - 127s 618ms/step - loss: 0.6923 - accuracy: 0.5223 - val_loss: 0.6850 - val_accuracy: 0.5360
Epoch 2/10
110/110 [==============================] - 65s 590ms/step - loss: 0.6616 - accuracy: 0.6072 - val_loss: 0.6365 - val_accuracy: 0.6520
Epoch 3/10
110/110 [==============================] - 67s 611ms/step - loss: 0.5526 - accuracy: 0.7217 - val_loss: 0.6590 - val_accuracy: 0.6350
Epoch 4/10
110/110 [==============================] - 67s 612ms/step - loss: 0.4327 - accuracy: 0.8068 - val_loss: 0.7164 - val_accuracy: 0.6510
Epoch 5/10
110/110 [==============================] - 67s 611ms/step - loss: 0.3090 - accuracy: 0.8744 - val_loss: 0.8783 - val_accuracy: 0.6410
Epoch 6/10
110/110 [==============================] - 68s 614ms/step - loss: 0.2099 - accuracy: 0.9238 - val_loss: 1.0806 - val_accuracy: 0.6340
Epoch 7/10
110/110 [==============================] - 65s 593ms/step - loss: 0.1422 - accuracy: 0.9496 - val_loss: 1.3435 - val_a

# Saving the model weights

In [ ]:
classifier.save_weights('SIMBArt-phrase.h5')

## Testing it out on the test dataset from the loaded weights

In [ ]:
loaded_classifier = BERTForClassification(model, num_classes=2)
loaded_classifier.compile(
    optimizer=tf.keras.optimizers.AdamW(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
dummy_input = tf.constant([[0, 0, 0, 0, 0]])
loaded_classifier(dummy_input)

loaded_classifier.load_weights('SIMBArt-phrase.h5')
loaded_classifier.evaluate(test_dataset)

32/32 [==============================] - 13s 170ms/step - loss: 1.6618 - accuracy: 0.6490


[1.6618105173110962, 0.6489999890327454]

#SIMBArt-sentence: Sentence-pair similarity detection BERT model trained on the PAWS dataset

Note: This section does the exact same data preprocessing steps and training parameters as above so I will not be going into detail with each step

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

model = TFAutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
inputs = tokenizer(['hello world','How are you'], padding=True, truncation=True, return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0],
       [ 101, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
output = model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 5, 768), dtype=float32, numpy=
array([[[-0.1688832 ,  0.13606352, -0.13940024, ..., -0.6251128 ,
          0.05217276,  0.36714536],
        [-0.3632746 ,  0.14121912,  0.87998706, ...,  0.10432965,
          0.28875774,  0.37267962],
        [-0.6985944 , -0.698797  ,  0.06450319, ..., -0.22103733,
          0.00986863, -0.5939796 ],
        [ 0.8309826 ,  0.12366714, -0.15119047, ...,  0.1030959 ,
         -0.6779267 , -0.26285207],
        [-0.40266573, -0.01928236,  0.57325023, ..., -0.20656858,
          0.02338598,  0.20126319]],

       [[ 0.0774727 ,  0.04501351, -0.16765113, ..., -0.33463413,
          0.16396382,  0.12080589],
        [-0.02718062, -0.7117558 , -0.29173565, ..., -0.12723008,
          0.6787842 , -0.53435344],
        [ 0.64420474, -0.98045236, -0.03809451, ..., -0.6010789 ,
          0.16867961, -0.49630678],
        [ 0.01914815, -0.7620821 ,  0.588272  , ..., -0.32224914

In [ ]:
dataset = load_dataset("paws", "labeled_final")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [ ]:
def combine_columns(example):
    return {'text': f"[{example['sentence1']}] [{example['sentence2']}]"}

dataset['train'] = dataset['train'].map(combine_columns)
dataset['validation'] = dataset['validation'].map(combine_columns)
dataset['test'] = dataset['test'].map(combine_columns)

dataset['train'] = dataset['train'].remove_columns(['sentence1', 'sentence2','id'])
dataset['validation'] = dataset['validation'].remove_columns(['sentence1', 'sentence2','id'])
dataset['test'] = dataset['test'].remove_columns(['sentence1', 'sentence2','id'])


Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][4]

{'label': 1,
 'text': '[It is the seat of Zerendi District in Akmola Region .] [It is the seat of the district of Zerendi in Akmola region .]'}

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

In [ ]:
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)
dataset_encoded

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
})

In [ ]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

dataset_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for val set ...
val_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['validation'][:])
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... and for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
inp, out = next(iter(train_dataset))
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 146), dtype=int64, numpy=
array([[ 101, 1031, 1996, ...,    0,    0,    0],
       [ 101, 1031, 2340, ...,    0,    0,    0],
       [ 101, 1031, 8183, ...,    0,    0,    0],
       ...,
       [ 101, 1031, 1996, ...,    0,    0,    0],
       [ 101, 1031, 2002, ...,    0,    0,    0],
       [ 101, 1031, 1996, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(64, 146), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 146), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[1 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 0 0 1

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, num_classes=2)

classifier.compile(
    optimizer=tf.keras.optimizers.AdamW(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history1 = classifier.fit(train_dataset, epochs=3, validation_data=val_dataset)

Epoch 1/3
772/772 [==============================] - 548s 639ms/step - loss: 0.6906 - accuracy: 0.5508 - val_loss: 0.6863 - val_accuracy: 0.5640
Epoch 2/3
772/772 [==============================] - 482s 624ms/step - loss: 0.6099 - accuracy: 0.6391 - val_loss: 0.3398 - val_accuracy: 0.8525
Epoch 3/3
772/772 [==============================] - 481s 623ms/step - loss: 0.2702 - accuracy: 0.8865 - val_loss: 0.2527 - val_accuracy: 0.9041


In [ ]:
classifier.save_weights('SIMBArt-sentence.h5')

In [ ]:
loaded_classifier = BERTForClassification(model, num_classes=2)
loaded_classifier.compile(
    optimizer=tf.keras.optimizers.AdamW(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
dummy_input = tf.constant([[0, 0, 0, 0, 0]])
loaded_classifier(dummy_input)

loaded_classifier.load_weights('SIMBArt-sentence.h5')
loaded_classifier.evaluate(test_dataset)

125/125 [==============================] - 27s 170ms/step - loss: 0.2650 - accuracy: 0.9016


[0.2650095820426941, 0.9016249775886536]

# <ins>Conclusion:</ins> As we can see, BERT model has significantly outperformed my initial mlp classifier in both the tasks. With phrase similarity having minimal improvement(likely due to dataset being small) and sentence similarity with 90% accuracy! Hence transformer models are much better at catching the contextual nuances of language compared to traditional models or algorithms.